**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Sep 12 03:23:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------



---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 b_depressed.csv
'Colab Notebooks'
 data_img
 face.jpg
 geological_similarity
'Getting started.pdf'
 IMG_20180605_063830.jpg
 IMG_20180605_080407.jpg
 IMG_20180823_181538.JPG
 IMG_20180905_124956.jpg
 IMG_20180909_164653.jpg
 IMG_20180909_182006.jpg
 IMG_20180913_114809.jpg
 IMG_20180913_144912.jpg
 IMG_20180913_193312.jpg
 IMG_20180913_193313.jpg
 mask_detection_lite.h5
 mask_detection_vgg.h5
 mnv.pdf
'M. SC '
'M. SC. Friends '
'photos my.pdf'
'STUDY OF TIME MINIMIZING TRANSPORTATION PROBLEM AND ASSIGNMENT.gdoc'
 Test_img_file.zip
 Train_img.zip
 yolov3
 yolov3_80_objects.weights
 yolov3_plate_detection


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14291, done.
remote: Total 14291 (delta 0), reused 0 (delta 0), pack-reused 14291
Receiving objects: 100% (14291/14291), 12.84 MiB | 20.19 MiB/s, done.
Resolving deltas: 100% (9749/9749), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo -e "mask\nmask_50\nno_mask" > data/obj.names
!echo -e "classes= 3\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3" > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-09-12 03:27:33--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  4.25MB/s    in 49s     

2020-09-12 03:28:22 (3.19 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/Mask_1.jpg     
  inflating: data/obj/Mask_1.txt     
  inflating: data/obj/Mask_10.jpg    
  inflating: data/obj/Mask_10.txt    
  inflating: data/obj/Mask_100.jpg   
  inflating: data/obj/Mask_100.txt   
  inflating: data/obj/Mask_101.jpg   
  inflating: data/obj/Mask_101.txt   
  inflating: data/obj/Mask_102.jpg   
  inflating: data/obj/Mask_102.txt   
  inflating: data/obj/Mask_103.jpg   
  inflating: data/obj/Mask_103.txt   
  inflating: data/obj/Mask_104.jpg   
  inflating: data/obj/Mask_104.txt   
  inflating: data/obj/Mask_105.jpg   
  inflating: data/obj/Mask_105.txt   
  inflating: data/obj/Mask_106.jpg   
  inflating: data/obj/Mask_106.txt   
  inflating: data/obj/Mask_107.jpg   
  inflating: data/obj/Mask_107.txt   
  inflating: data/obj/Mask_108.jpg   
  inflating: data/obj/Mask_108.txt   
  inflating: data/obj/Mask_109.jpg   
  inflating: data/obj/Mask_109.txt   
  inflating: data/obj/Mask_11.jpg    
  inflating: 

In [13]:
import os

In [14]:
os.getcwd()

'/content/darknet'

In [15]:
os.listdir('/content/darknet/data/obj')

['Mask_548.txt',
 'Mask_186.jpg',
 'Mask_241.txt',
 '32vp.jpg',
 'Mask_316.jpg',
 'Mask_43.txt',
 'Mask_443.jpg',
 'Mask_273.jpg',
 'Mask_228.txt',
 'Mask_7.txt',
 '31vv.jpg',
 'Mask_121.jpg',
 '71fD85n0DuL._SL1496_.jpg',
 'Mask_51.jpg',
 'Mask_254.jpg',
 'Mask_100.jpg',
 'Mask_98.jpg',
 'Mask_610.jpg',
 'Mask_236.txt',
 'Mask_598.jpg',
 'Mask_572.txt',
 'Mask_422.txt',
 'xxz1.jpg',
 'Mask_410.txt',
 '2vj.txt',
 'Mask_326.jpg',
 'Mask_406.jpg',
 '4.jpg',
 'Mask_583.jpg',
 'Mask_465.txt',
 'Mask_291.txt',
 '33vj.txt',
 'Mask_177.jpg',
 'Mask_493.jpg',
 'Mask_22.jpg',
 '5.jpg',
 'Mask_511.jpg',
 'Mask_369.txt',
 'AFP_1PL0FN_1583291348242_1583402523513.jpg',
 'Mask_24.txt',
 'Mask_148.jpg',
 'Mask_398.txt',
 'Mask_519.txt',
 'Mask_328.jpg',
 'Mask_484.txt',
 'Mask_562.jpg',
 'Mask_567.txt',
 'Mask_467.jpg',
 '5vj.txt',
 'Mask_619.txt',
 'Mask_157.txt',
 'Mask_130.txt',
 'Mask_672.txt',
 'Mask_26.jpg',
 'Mask_295.txt',
 '103vv.jpg',
 '53vv.jpg',
 'Mask_195.txt',
 'Mask_146.jpg',
 'Mask_579

In [16]:

# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths= glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
0 0.45416666666666666 0.4772727272727273 0.04861111111111111 0.08409090909090909

['0', '0.45416666666666666', '0.4772727272727273', '0.04861111111111111', '0.08409090909090909']
621 data/obj/Mask_368.txt
0 0.45416666666666666 0.4772727272727273 0.04861111111111111 0.08409090909090909
0 0.43506944444444445 0.25625 0.014583333333333334 0.060227272727272727

['0', '0.43506944444444445', '0.25625', '0.014583333333333334', '0.060227272727272727']
621 data/obj/Mask_368.txt
0 0.43506944444444445 0.25625 0.014583333333333334 0.060227272727272727
0 0.4791666666666667 0.2755681818181818 0.027777777777777776 0.060227272727272727

['0', '0.4791666666666667', '0.2755681818181818', '0.027777777777777776', '0.060227272727272727']
621 data/obj/Mask_368.txt
0 0.4791666666666667 0.2755681818181818 0.027777777777777776 0.060227272727272727
0 0.5270833333333333 0.26931818181818185 0.030555555555555555 0.06818181818181818

['0', '0.5270833333333333', '0.2

In [17]:
len(txt_file_paths)

800

In [18]:
import glob
images_list = glob.glob("data//obj//*.jpg")
print(images_list)

['data//obj/Mask_186.jpg', 'data//obj/32vp.jpg', 'data//obj/Mask_316.jpg', 'data//obj/Mask_443.jpg', 'data//obj/Mask_273.jpg', 'data//obj/31vv.jpg', 'data//obj/Mask_121.jpg', 'data//obj/71fD85n0DuL._SL1496_.jpg', 'data//obj/Mask_51.jpg', 'data//obj/Mask_254.jpg', 'data//obj/Mask_100.jpg', 'data//obj/Mask_98.jpg', 'data//obj/Mask_610.jpg', 'data//obj/Mask_598.jpg', 'data//obj/xxz1.jpg', 'data//obj/Mask_326.jpg', 'data//obj/Mask_406.jpg', 'data//obj/4.jpg', 'data//obj/Mask_583.jpg', 'data//obj/Mask_177.jpg', 'data//obj/Mask_493.jpg', 'data//obj/Mask_22.jpg', 'data//obj/5.jpg', 'data//obj/Mask_511.jpg', 'data//obj/AFP_1PL0FN_1583291348242_1583402523513.jpg', 'data//obj/Mask_148.jpg', 'data//obj/Mask_328.jpg', 'data//obj/Mask_562.jpg', 'data//obj/Mask_467.jpg', 'data//obj/Mask_26.jpg', 'data//obj/103vv.jpg', 'data//obj/53vv.jpg', 'data//obj/Mask_146.jpg', 'data//obj/Mask_579.jpg', 'data//obj/38vj.jpg', 'data//obj/Mask_140.jpg', 'data//obj/Mask_420.jpg', 'data//obj/7vp.jpg', 'data//obj/Mask

In [19]:
len(images_list)

799

In [20]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [21]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000021, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.001484, iou_loss = 0.000000, total_loss = 0.001484 
 total_bbox = 271368, rewritten_bbox = 0.073332 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.673859, GIOU: 0.672891), Class: 0.999047, Obj: 0.848530, No Obj: 0.000746, .5R: 1.000000, .75R: 0.000000, count: 1, class_loss = 0.035130, iou_loss = 0.090221, total_loss = 0.125351 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.749825, GIOU: 0.744984), Class: 0.997382, Obj: 0.548500, No Obj: 0.000852, .5R: 1.000000, .75R: 0.750000, count: 4, class_loss = 0.448946, iou_loss = 0.400560, total_loss = 0.849505 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.767195, GIOU: 0.753974), Class: 0.996979, Obj: 0.710308, No Obj